In [13]:

from google.oauth2 import service_account
import pandas as pd
from googleapiclient.discovery import build
# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/Users/linabattikha/git/book_blog/eda/credentials.json'

# Define the required scopes (read-only for sheets)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Authenticate using service account credentials
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Create the Google Sheets API client
service = build('sheets', 'v4', credentials=creds)

# The ID of the Google Sheet
SPREADSHEET_ID = '1wRpqF_aaRh9SVh7NTkeyF84Ripzh45pfy3DWYpMgnwI'

# Define the range of cells to read (e.g., 'Sheet1!A1:D100')
RANGE_NAME = 'Sheet1!A:H'

# Call the Sheets API to fetch data from the sheet
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()

# Extract rows from the result
rows = result.get('values', [])

# Load the rows into a pandas DataFrame
if rows:
    # The first row should be treated as the header (column names)
    df = pd.DataFrame(rows[1:], columns=rows[0])
    print(df)
else:
    print('No data found.')

print("END")


                    Title           Author              Genre   
0          Grayson's Vow       Mia Sheridan           Romance  \
1            The Contract  Melanie Moreland           Romance   
2               The Favor    Suzanne Wright           Romance   
3                   Drago     Sarah Brianne           Romance   
4            Twisted Hate        Ana Huang            Romance   
..                    ...               ...               ...   
65   The Paradise Problem  Christina Lauren           Romance   
66               Redeemed      Lauren Asher           Romance   
67     The Night Blossoms      Leylah Attar          Mystery    
68  The Midnight Library          Matt Haig  Magical Realism    
69      The Silent Waters  Brittainy Cherry           Romance   

               Trope  Data Finished Rating   
0       Fake Marriage       8/20/22      4  \
1       Fake Marriage   August 2022      5   
2       Fake Marriage   August 2022      5   
3   Arranged Marriage      12/14/22

In [14]:
df.head()

,Title,Author,Genre,Trope,Data Finished,Rating,Notes,Description - ChatGpt
0,Grayson's Vow,Mia Sheridan,Romance,Fake Marriage,8/20/22,4,"After reading Archer's Voice, I had set the ba...",Marriage of convenience where the female lead ...
1,The Contract,Melanie Moreland,Romance,Fake Marriage,August 2022,5,I absolutely loved this book. The author did a...,A ruthless boss and his fed-up assistant find ...
2,The Favor,Suzanne Wright,Romance,Fake Marriage,August 2022,5,Absolutely loved this book. The first book tha...,Vienna Stratton reluctantly accepted help from...
3,Drago,Sarah Brianne,Romance,Arranged Marriage,12/14/22,2,Did not like this book at all. There was poten...,"In a gritty love story, Drago, the Caruso fami..."
4,Twisted Hate,Ana Huang,Romance,Enemies to Lovers,12/11/22,4,This book was a good meh. I really liked the c...,"Josh can't stand Jules, but their fiery animos..."


In [15]:
df.columns = [i.strip () for i in df.columns]

In [16]:
romance_only = df[df["Genre"] == "Romance"]

In [17]:
romance_only.groupby("Trope").count()

,Title,Author,Genre,Data Finished,Rating,Notes,Description - ChatGpt
Trope,,,,,,,
,1,1,1,1,1,1,1
Alpha male,2,2,2,2,2,2,2
Arranged Marriage,2,2,2,2,2,2,2
Bestfriend's Brother,3,3,3,3,3,3,3
Children in Tow,1,1,1,1,1,1,1
Children in Tow,2,2,2,2,2,2,2
Enemies to Lovers,3,3,3,3,3,3,3
Fake Marriage,3,3,3,3,3,3,3
Fake Marriage,1,1,1,1,1,1,1
